In [23]:
import sys
sys.path.append('')
import torch
from maskAttention import *
from predictionModel import *
from gle import *
import configurationGIN as cfg
import argparse


In [25]:
parser = argparse.ArgumentParser()

parser.add_argument("--nb_classes", type=int, default=1, help="The number of classes to predict for our clf task")
# This is automatically set from the dataset module
parser.add_argument("--gnn_node_features", type=int, default=3, help="Dim of features in the nodes that we will feed to GNN")

# GNN model parameters
# -- pooling
parser.add_argument("--gnn_pooling_cls_name", type=str, default="ConfigurablePooling", help="One of pooling layers define din 'cardio.networks.gnn' ")
parser.add_argument("--pool_only_lesion_points", action='store_true', help="In case we use whole artery, gnn pooling will pool only lesion points")
parser.add_argument("--node_pooling", type=str, default='mean', help="How to pool the nodes of the gnn: sum, mean, max, all_stats")
# -- model
parser.add_argument("--gnn_is_siamese", default=False, help="In case we use siamese dataset use this flag")
parser.add_argument("--gnn_cls_name", type=str, default="GIN", help="One of the pred class names in 'torch_geometric.nn.models.basic_gnn' or one of our custom gnn class names, CustomGIN")
parser.add_argument("--gnn_node_emb_dim", type=int, default=256, help="GNN hidden dim that will be input of the GATconv")
parser.add_argument("--gnn_global_hidden_dim", type=int, default=512, help="GNN The dimension to have after the pooling ")
parser.add_argument("--gnn_nb_layers", type=int, default=6, help="GNN number of Transformer layers")
parser.add_argument("--gnn_dropout", type=float, default=0.415081799988646554, help="GNN The dropout on the Attention layers of the gnn")
parser.add_argument("--gnn_act", type=str, default="relu", help="The type of activation (one of torch activations)")
parser.add_argument("--gnn_norm", type=str, default="BatchNorm", help="One of the normalizations, see 'torch_geometric.nn.norm.__init__.py'")
parser.add_argument("--gnn_freeze_weights", type=bool, default=False, help="Freeze gnn weights (not pooling)")
parser.add_argument("--gnn_ffr_pooling_factor", type=int, default=1, help="Scale the global dim by a factor multiple times (until dim is >= 16) before you concat it with ffr")
parser.add_argument("--gnn_ffr_pooling_proj_dim", type=int, default=16, help="Stops scalling when you reach this projection dimention")
parser.add_argument("--use_lesion_wide_info", default=True,  help="Uses lesion wide features like FFR and DS")
# parser.add_argument("--nb_clinical_data_features", default=24, type=int, help="The number of clinical data features, if used with ClinicalDataPooling")
parser.add_argument("--lesion_wide_feat_dim", default=21, type=int, help="The number of clinical data features, if used with ClinicalDataPooling")
# parser.add_argument("--gnn_pooling_cls_name", type=str, default="ConfigurablePooling", help="One of pooling layers define din 'cardio.networks.gnn' ")

confs = parser.parse_args(args=[])

backbone = GNNClassificationModule(confs)
print(backbone)

GNNClassificationModule(
  (model): GNNClassifier(
    (gnn): GIN(3, 256, num_layers=6)
    (polling): ConfigurablePooling(
      (lin1): Sequential(
        (0): Linear(in_features=256, out_features=256, bias=True)
        (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.4150817999886465, inplace=False)
        (4): Linear(in_features=256, out_features=128, bias=True)
        (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Dropout(p=0.4150817999886465, inplace=False)
        (8): Linear(in_features=128, out_features=64, bias=True)
        (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU()
        (11): Dropout(p=0.4150817999886465, inplace=False)
      )
      (lin2): Sequential(
        (0): Linear(in_features=85, out_features=32, bias=True)
        (1): BatchNorm1d(32, eps=1e-05

In [33]:
confs.lesionNum =9
confs.fusion_method ='atten'

patient_model=mergModule(backbone, confs,lesionNum=confs.lesionNum, embedding_dim=32,dim_feedforward=32, nhead=4,num_layers=4,qkv_bias=False,drop_rate=0.3)
print(patient_model)

mergModule(
  (enc): GNNClassificationModule(
    (model): GNNClassifier(
      (gnn): GIN(3, 256, num_layers=6)
      (polling): ConfigurablePooling(
        (lin1): Sequential(
          (0): Linear(in_features=256, out_features=256, bias=True)
          (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Dropout(p=0.4150817999886465, inplace=False)
          (4): Linear(in_features=256, out_features=128, bias=True)
          (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): ReLU()
          (7): Dropout(p=0.4150817999886465, inplace=False)
          (8): Linear(in_features=128, out_features=64, bias=True)
          (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (10): ReLU()
          (11): Dropout(p=0.4150817999886465, inplace=False)
        )
        (lin2): Sequential(
          (0): Linear(in_features=85, out_fea